## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [12]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
#example沒有輸入Activation，記得加上去

In [28]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
print(x_test.shape)

## Normalize Data, 如果是競賽的時候會把RGB分開
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 
print(mean_train)
print(x_train.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
120.70756512369792
(50000, 32, 32, 3)


In [16]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()
print(y_test[0])
print(y_test[1])

[1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0.]


In [15]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), padding='same', input_shape=x_train.shape[1:]))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))


#卷積組合
classifier.add(Convolution2D(64, (3,3), input_shape=x_train.shape[1:]))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(256)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=3)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/3
50000/50000 [==============================] - 196s 4ms/step - loss: 4.2938 - accuracy: 0.3842
Epoch 2/3
50000/50000 [==============================] - 201s 4ms/step - loss: 1.4953 - accuracy: 0.5228
Epoch 3/3
50000/50000 [==============================] - 189s 4ms/step - loss: 1.1192 - accuracy: 0.6163


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [33]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) #zero是全黑的圖，不是有意義的圖，但還是有一個最像的
classifier.predict(input_example) #預測的圖片要跟被訓練的圖片一樣大
#被測試的圖片也要經過normalization
#print(np.zeros(shape=(8,8,3))) #有8個8*3的陣列
#print(input_example)
y_predict = classifier.predict(x_test[:5])
print(y_predict)

for i in y_predict:
    print(np.where(i == np.max(i)))

[[1.25087984e-03 1.22035730e-04 3.49712442e-03 4.70637351e-01
  1.15916028e-03 2.45054606e-02 5.58651462e-02 1.34805078e-03
  4.40180868e-01 1.43390161e-03]
 [3.53596956e-02 6.99509025e-01 8.58351450e-06 5.14643148e-07
  6.65605910e-07 1.09459943e-08 2.48606824e-07 3.21147610e-07
  2.58845747e-01 6.27523055e-03]
 [3.92809600e-01 4.08425480e-02 1.61274392e-02 2.10533221e-03
  7.53572094e-04 6.10485076e-05 3.25425674e-04 1.25126098e-03
  5.32801747e-01 1.29219554e-02]
 [1.81096390e-01 1.13809863e-02 3.89419049e-02 1.09155243e-03
  2.07166909e-03 1.82810563e-05 1.38690718e-03 1.70532541e-04
  7.60710418e-01 3.13132233e-03]
 [1.49511527e-06 1.15369885e-05 4.08349093e-03 1.34786600e-02
  6.09457016e-01 1.98960421e-03 3.70904267e-01 3.67503126e-05
  3.28258539e-05 4.27245368e-06]]
(array([3]),)
(array([1]),)
(array([8]),)
(array([8]),)
(array([4]),)


In [30]:
a = np.arange(1,193).reshape(2,4,8,3)
print(a)

mean = np.mean(a,axis=(0,1,2,3)) #axis的維度跟shape是一樣的
print('1')
print(mean)

mean = np.mean(a,axis=0)
print('2')
print(mean)

mean = np.mean(a,axis=1) #axis為1代表的是reshape時'4'的那個維度, 所以37=1+25+49+73
print('3')
print(mean)

[[[[  1   2   3]
   [  4   5   6]
   [  7   8   9]
   [ 10  11  12]
   [ 13  14  15]
   [ 16  17  18]
   [ 19  20  21]
   [ 22  23  24]]

  [[ 25  26  27]
   [ 28  29  30]
   [ 31  32  33]
   [ 34  35  36]
   [ 37  38  39]
   [ 40  41  42]
   [ 43  44  45]
   [ 46  47  48]]

  [[ 49  50  51]
   [ 52  53  54]
   [ 55  56  57]
   [ 58  59  60]
   [ 61  62  63]
   [ 64  65  66]
   [ 67  68  69]
   [ 70  71  72]]

  [[ 73  74  75]
   [ 76  77  78]
   [ 79  80  81]
   [ 82  83  84]
   [ 85  86  87]
   [ 88  89  90]
   [ 91  92  93]
   [ 94  95  96]]]


 [[[ 97  98  99]
   [100 101 102]
   [103 104 105]
   [106 107 108]
   [109 110 111]
   [112 113 114]
   [115 116 117]
   [118 119 120]]

  [[121 122 123]
   [124 125 126]
   [127 128 129]
   [130 131 132]
   [133 134 135]
   [136 137 138]
   [139 140 141]
   [142 143 144]]

  [[145 146 147]
   [148 149 150]
   [151 152 153]
   [154 155 156]
   [157 158 159]
   [160 161 162]
   [163 164 165]
   [166 167 168]]

  [[169 170 171]
   [172 173 174